In [18]:
from sgRNAutils import revComp
revComp('ATG')

'CAT'

In [19]:
from sgRNAutils import refSeq
refSeqPerChromosome= refSeq()
refSeqPerChromosome['X'][20000080:20000090]

Seq('TGCTAGAGGG')

In [17]:
# This cell takes about 2 min to run but you can avoid it by running the next cell and loading the output from this cell from MockMaterials
from sgRNAutils import make_dataframe_from_TFs_list
TFsdf = make_dataframe_from_TFs_list("inputfiles/TFs.xlsx", refSeqPerChromosome)
display(TFsdf)

KeyboardInterrupt: 

In [20]:
from sgRNAutils import make_homology_arm_fragments
import pandas as pd
TFsdf = pd.read_csv("inputfiles/mockMaterials/TFsdfTruncated.csv")
TFsdf = make_homology_arm_fragments(TFsdf, refSeqPerChromosome)
TFsdf = TFsdf.reset_index()
del TFsdf["index"]
del TFsdf["ID"]
display(TFsdf)
print(len(TFsdf.iloc[0]["HAL"]))
print(len(TFsdf.iloc[0]["HAR"]))
print(str(refSeqPerChromosome["2L"][491316-1:491318]))
print(len(str(refSeqPerChromosome["2L"][491316-1:491318])))

,Gene_ID,Transcript_ID,Chromosome,Gene_Region,Start,Stop,Strand,Reference_Seq,HAL,HAR
0,FBgn0003963,FBtr0333013,2L,start_codon,491316,491318,+,CACTGATAAAAAATACCCTAAATTATCGCAACGCAAATGGTAAAGT...,TTCGAGTCCAGCTGCTGGACCGGAGGTATATCTTCAGCTACGTCGG...,TCAGATGGGAAAAGCGATCGTGATTAGTGGAGAGTTACGAGATAGC...
1,FBgn0020309,FBtr0335474,2L,start_codon,11803437,11803439,-,TTTCTCCCGCTCTGCGCTCTCCTCGATGTTTGCTGAATTATTTGCT...,CAGCACGTGAGCGCTGCCAGCTCGGTGCCATCAGTAGTAACTCCTG...,CGAAAGCAGTCGACAGCAGCAACAACAGCAATATGTAGGCGCTCCT...
2,FBgn0263102,FBtr0088275,2R,start_codon,10561805,10561807,+,TCGTCAAATGCAAAAGCAACAAATCGAGGCACGTTGCTGCACCGGA...,TTCGTATGGTTTTTCGTATTCGTTTCACGCCGAATCCTAGCTGAAT...,GGACGACCTGGAGTTTTTGAACAAAAGTTTGATTAGCTTCATAACT...
3,FBgn0016694,FBtr0332081,3L,stop_codon,7816326,7816328,-,GTGTGAATTGTCCTATTGACGAATTCAGTCAAAAACTTATGCACTT...,TGATAAATACAATTTTGTGACTTGGCCTGAGGACACCACAACAATT...,GCGATCCCGCGATGCCCGGCGCCAGAAGGAGAACCAGATTGCGATG...
4,FBgn0031375,FBtr0330202,2L,stop_codon,1950795,1950797,-,CCTGCTGTACTATGCACCCAATCAGCTGATGTGCGCCGCTGCTGCC...,GCAGAAGGACCCCTGTCCACCTCCACCCACCCCGCAGAACCTCAGC...,ACGGGGTCGACCTCGACGACCAGCTCCACGGTGACCACCACCACCA...


225
225
ATG
3


In [21]:
# this showes that there are not only conventional start and stop codons e.g. CAT, TCA
for index, row in TFsdf.iterrows():
    print(refSeqPerChromosome[row["Chromosome"]][row['Start']-1:row['Stop']], row['Start']-1, row['Stop'])

ATG 491315 491318
CAT 11803436 11803439
ATG 10561804 10561807
TTA 7816325 7816328
CTA 1950794 1950797


In [22]:
from sgRNAutils import gRNA_stringencyIterator
#For row 5 of transcription factor start/stop site dataframe, select a guideRNA
#Row 1 is start gene(-) sgRNA(+), takes 1 min to run
#Row 0 is start gene(+) sgRNA(-), takes 5.3 min to run

print(f"Selecting guide RNA for TFsdf row {0}")
window = 21
filtered_sgRNA_0 = gRNA_stringencyIterator(TFsdf.iloc[0], window, refSeqPerChromosome)
display(filtered_sgRNA_0)
# test whether sgRNA within window
for i in range(len(filtered_sgRNA_0)):
    if filtered_sgRNA_0['fmin'].iloc[i] >= filtered_sgRNA_0["Start"].iloc[i] - window & filtered_sgRNA_0['fmax'].iloc[i] <= filtered_sgRNA_0["Stop"].iloc[i] + window:
        print("sgRNA within window")

print(f"Selecting guide RNA for TFsdf row {1}")
filtered_sgRNA_1 = gRNA_stringencyIterator(TFsdf.iloc[1], window, refSeqPerChromosome)
display(filtered_sgRNA_1)
# test whether sgRNA within window
for i in range(len(filtered_sgRNA_1)):
    if filtered_sgRNA_1['fmin'].iloc[i] >= filtered_sgRNA_1["Start"].iloc[i] - window & filtered_sgRNA_1['fmax'].iloc[i] <= filtered_sgRNA_1["Stop"].iloc[i] + window:
        print("sgRNA within window")

Selecting guide RNA for TFsdf row 0
stringency 0
stringency 1
stringency 2
stringency 3
stringency 4


,fmin,fmax,#chr,strand,sgRNA_sequence,Gene_ID,Transcript_ID,Chromosome,Gene_Region,Start,Stop,Strand,Reference_Seq,HAL,HAR
0,491298,491320,2L,-,CCAGCAGCTGGACTCGAAATGTC,FBgn0003963,FBtr0333013,2L,start_codon,491316,491318,+,CACTGATAAAAAATACCCTAAATTATCGCAACGCAAATGGTAAAGT...,TTCGAGTCCAGCTGCTGGACCGGAGGTATATCTTCAGCTACGTCGG...,TCAGATGGGAAAAGCGATCGTGATTAGTGGAGAGTTACGAGATAGC...


sgRNA within window
Selecting guide RNA for TFsdf row 1
stringency 0
stringency 1


,fmin,fmax,#chr,strand,sgRNA_sequence,Gene_ID,Transcript_ID,Chromosome,Gene_Region,Start,Stop,Strand,Reference_Seq,HAL,HAR
0,11803435,11803457,2L,+,TGCATTCTTGTGTCCAGATCCGG,FBgn0020309,FBtr0335474,2L,start_codon,11803437,11803439,-,TTTCTCCCGCTCTGCGCTCTCCTCGATGTTTGCTGAATTATTTGCT...,CAGCACGTGAGCGCTGCCAGCTCGGTGCCATCAGTAGTAACTCCTG...,CGAAAGCAGTCGACAGCAGCAACAACAGCAATATGTAGGCGCTCCT...
1,11803436,11803458,2L,+,GCATTCTTGTGTCCAGATCCGGG,FBgn0020309,FBtr0335474,2L,start_codon,11803437,11803439,-,TTTCTCCCGCTCTGCGCTCTCCTCGATGTTTGCTGAATTATTTGCT...,CAGCACGTGAGCGCTGCCAGCTCGGTGCCATCAGTAGTAACTCCTG...,CGAAAGCAGTCGACAGCAGCAACAACAGCAATATGTAGGCGCTCCT...


sgRNA within window
sgRNA within window


In [6]:
from sgRNAutils import sgRNApositionCheck
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
minDistance = -24
maxDistance = 21
checked_0 = sgRNApositionCheck(filtered_sgRNA_0, minDistance, maxDistance)
checked_1 = sgRNApositionCheck(filtered_sgRNA_1, minDistance, maxDistance)
print(checked_0)
print(checked_1)

     fmin    fmax #chr strand           sgRNA_sequence      Gene_ID  \
0  491298  491320   2L      -  CCAGCAGCTGGACTCGAAATGTC  FBgn0003963   

  Transcript_ID Chromosome  Gene_Region   Start    Stop Strand  \
0   FBtr0333013         2L  start_codon  491316  491318      +   

                                       Reference_Seq  \
0  CACTGATAAAAAATACCCTAAATTATCGCAACGCAAATGGTAAAGT...   

                                                 HAL  \
0  TTCGAGTCCAGCTGCTGGACCGGAGGTATATCTTCAGCTACGTCGG...   

                                                 HAR  positionScore CDS_side  \
0  TCAGATGGGAAAAGCGATCGTGATTAGTGGAGAGTTACGAGATAGC...              2      HAR   

  non_CDS_side PAM_in_start/stop max_15_bp_3’_overhang PAM_in_CDS  \
0          HAL             False                 False      False   

  PAM_outside_CDS SRS_in_CDS  \
0            True      False   

                                        CDS_boundary  \
0  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   

                    

In [7]:
from sgRNAutils import checkCDSCutandOrder
sgRNA = checkCDSCutandOrder(checked_1)
display(sgRNA)

'TGCATTCTTGTGTCCAGATCCGG'

In [8]:
# for testing print the HA in CDS
HA_0 = checked_0.iloc[0][checked_0.iloc[0]["CDS_side"]]
display(HA_0)
print(len(HA_0))
HA_1 = checked_1.iloc[0][checked_1.iloc[0]["CDS_side"]]
display(HA_1)
print(len(HA_1))    

'TCAGATGGGAAAAGCGATCGTGATTAGTGGAGAGTTACGAGATAGCTTTGTAATGACAAAGTGTAATTTACAAAAGTGCATTTGTTCGTTGAGGCAAAATAATGGTTGCAGATAACTGCACAGGTGCTATAAAAAGAAGGTCGTCGACTAAACCCCGACTTGATAAAACTAAACTTGTAGCTCACCTTTATTCAACGGCTTCGGGTTGGACTGTTTGCGTCGTGA'

225


'CAGCACGTGAGCGCTGCCAGCTCGGTGCCATCAGTAGTAACTCCTGTTGTGACCACTGGTGGGACAACGATCACATTGGGCGGCCCACCACCGCTCCCTAAATCCGAGCACAAAGAGGATGGCAAGCCGCCGCACGGCATCGAAATGTACAAGGTGAACATTGAGGACATTTCGCAGCTCTTCACCTACCACGAGGTCTTTGGCAAAATCCACGGCGATGTGGTC'

225


In [9]:
from sgRNAutils import codonFragmenter
pd.set_option('display.max_column', None)
CDS_0, df_CDS_adjusted_0 = codonFragmenter(checked_0.iloc[0])
print(CDS_0)
CDS_1, df_CDS_adjusted_1 = codonFragmenter(checked_1.iloc[0])
print(CDS_1)

['TCA', 'GAT', 'GGG', 'AAA', 'AGC', 'GAT', 'CGT']
['TTT', 'GAT', 'GTG', 'GCC', 'ATC', 'CAC', 'GAC']


c:\Users\magd5549\OneDrive - Nexus365\Bioinformatics\AutoTagsCRISPR\sgRNAutils.py:496: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winnerdf["CDS_boundary_HA"] = np.array(winnerdf["CDS_boundary"]) - 1 #HA is 1bp to the right of stop
c:\Users\magd5549\OneDrive - Nexus365\Bioinformatics\AutoTagsCRISPR\sgRNAutils.py:496: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winnerdf["CDS_boundary_HA"] = np.array(winnerdf["CDS_boundary"]) - 1 #HA is 1bp to the right of stop
c:\Users\magd5549\OneDrive - Nexus365\Bioinformatics\AutoTagsCRISPR\sgRNAutils.py:500: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [10]:
from sgRNAutils import codonReverseFragmenter
df_HA_0_after = codonReverseFragmenter(CDS_0, df_CDS_adjusted_0)
HA_0_after = df_HA_0_after[df_HA_0_after["CDS_side"]]
display(HA_0_after)
display(HA_0)
print(len(HA_0_after))
if HA_0_after == HA_0:
    print("all working")
else: print("not working")

df_HA_1_after = codonReverseFragmenter(CDS_1, df_CDS_adjusted_1)
HA_1_after = df_HA_1_after[df_HA_1_after["CDS_side"]]
display(HA_1_after)
display(HA_1)
print(len(HA_1_after))
if HA_1_after == HA_1:
    print("all working")
else: print("not working")

c:\Users\magd5549\OneDrive - Nexus365\Bioinformatics\AutoTagsCRISPR\sgRNAutils.py:586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winnerdf.at["HAR"] = CDS + winnerdf["HAR"][winnerdf["CDS_boundary_HA"][-1]+1:]


'TCAGATGGGAAAAGCGATCGTGATTAGTGGAGAGTTACGAGATAGCTTTGTAATGACAAAGTGTAATTTACAAAAGTGCATTTGTTCGTTGAGGCAAAATAATGGTTGCAGATAACTGCACAGGTGCTATAAAAAGAAGGTCGTCGACTAAACCCCGACTTGATAAAACTAAACTTGTAGCTCACCTTTATTCAACGGCTTCGGGTTGGACTGTTTGCGTCGTGA'

'TCAGATGGGAAAAGCGATCGTGATTAGTGGAGAGTTACGAGATAGCTTTGTAATGACAAAGTGTAATTTACAAAAGTGCATTTGTTCGTTGAGGCAAAATAATGGTTGCAGATAACTGCACAGGTGCTATAAAAAGAAGGTCGTCGACTAAACCCCGACTTGATAAAACTAAACTTGTAGCTCACCTTTATTCAACGGCTTCGGGTTGGACTGTTTGCGTCGTGA'

225
all working


c:\Users\magd5549\OneDrive - Nexus365\Bioinformatics\AutoTagsCRISPR\sgRNAutils.py:588: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winnerdf.at["HAL"] =  winnerdf["HAL"][:winnerdf["CDS_boundary_HA"][0]+1] + CDS


'CAGCACGTGAGCGCTGCCAGCTCGGTGCCATCAGTAGTAACTCCTGTTGTGACCACTGGTGGGACAACGATCACATTGGGCGGCCCACCACCGCTCCCTAAATCCGAGCACAAAGAGGATGGCAAGCCGCCGCACGGCATCGAAATGTACAAGGTGAACATTGAGGACATTTCGCAGCTCTTCACCTACCACGAGGTCTTTGGCAAAATCCACGGCGATGTGGTC'

'CAGCACGTGAGCGCTGCCAGCTCGGTGCCATCAGTAGTAACTCCTGTTGTGACCACTGGTGGGACAACGATCACATTGGGCGGCCCACCACCGCTCCCTAAATCCGAGCACAAAGAGGATGGCAAGCCGCCGCACGGCATCGAAATGTACAAGGTGAACATTGAGGACATTTCGCAGCTCTTCACCTACCACGAGGTCTTTGGCAAAATCCACGGCGATGTGGTC'

225
all working


In [11]:
from sgRNAutils import mutator_Marina
# formula to calculate base position to be mutated = base position relative to fmax + positionScore - CDS start
# to test try different base positions relative to fmax e.g. [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8]
base_position_relative_to_fmax = 4
if df_CDS_adjusted_0["Strand"] == '+':
    codonCoordinates = pd.read_excel("inputfiles/fmaxStopScoreML.xlsx", sheet_name="CodonCoordinatePlus", index_col= 0)
    print("plus")
elif df_CDS_adjusted_0["Strand"] == '-':
    codonCoordinates = pd.read_excel("inputfiles/fmaxStopScoreML.xlsx", sheet_name="CodonCoordinateMinus", index_col= 0)
    print("minus")
print(CDS_0)
print("positionScore:", df_CDS_adjusted_0["positionScore"])
print("CDS start:", df_CDS_adjusted_0["CDS_boundary"][0])
fragmentedDCS = mutator_Marina(base_position_relative_to_fmax,CDS_0,df_CDS_adjusted_0,codonCoordinates)
print(fragmentedDCS)

plus
['TCA', 'GAT', 'GGG', 'AAA', 'AGC', 'GAT', 'CGT']
positionScore: 2
CDS start: 1
['TCA', 'GAC', 'GGG', 'AAA', 'AGC', 'GAT', 'CGT']


In [12]:
def find_best_mutation(winnerdf):
    """
        In the case where a fragment needs to be mutated, will mutate in CDS (preferably PAM, if not then in the sgRNA). 
        If not possible, will mutate PAM outside of CDS to NTG/CTN.
        
        params:
            winnerdf: the pandas series of the winning sgRNA in format:
                fmin                                                            396165
                fmax                                                            396187
                #chr                                                                 X
                strand                                                               -
                sgRNA_sequence                                 CCGAGTGTGTTAATGAAAAATAA
                Gene_ID                                                    FBgn0004170
                Transcript_ID                                              FBtr0070073
                Chromosome                                                           X
                Gene_Region                                                start_codon
                Start                                                           396177
                Stop                                                            396179
                Strand                                                               +
                Reference_Seq        ACCTGCGATAATTTGACATTCTTAGAAACTACCTGAAGAAATAGGA...
                upstreamHA           TCTGGTCAGTGCCATACCCCTTGGTGTATACTTGCGAGTCTTAATT...
                downstreamHA         AACACACTCGGAGCTTTCTTTAACTTTCCGGATAACGATCAACAGA...
                positionScore                                                        8
                PAM_in_start/stop                                                False
                <15_bp3’_overhang                                                 True
                PAM_in_CDS                                                       False
                PAM_outside_CDS                                                   True
                CDS_boundary                                                       >22
                lastG                                                              8.0
                cutSite                                                           13.0
                mutated                                                          False

        """
    import pandas as pd

    if winnerdf["Strand"] == '+':
        codonCoordinates = pd.read_excel("inputfiles/fmaxStopScoreML.xlsx", sheet_name="CodonCoordinatePlus", index_col= 0)
    elif winnerdf["Strand"] == '-':
        codonCoordinates = pd.read_excel("inputfiles/fmaxStopScoreML.xlsx", sheet_name="CodonCoordinateMinus", index_col= 0)

    # Condition A: PAM in CDS
    if winnerdf["PAM_in_CDS"] == True and winnerdf["mutated?"] == False:
        fragmentedCDS, winnerdf = codonFragmenter(winnerdf, winnerdf["CDS_side"], winnerdf["CDS_boundary"])

        for basePosition in winnerdf["mutable_PAM"]:
            
            try: 
                fragmentedCDS = mutator_Marina(basePosition, fragmentedCDS, winnerdf, codonCoordinates, PAM=True)
                winnerdf["mutated?"] = True
                break

            except ValueError: continue

        winnerdf = codonReverseFragmenter(fragmentedCDS, winnerdf, winnerdf["CDS_side"], winnerdf["CDS_boundary"])

    # Condition B: sgRNA in CDS
    if winnerdf["SRS_in_CDS"] == True and winnerdf["mutated?"] == False:
        number_of_mutations = 0
        position = 0
        fragmentedCDS, winnerdf = codonFragmenter(winnerdf, winnerdf["CDS_side"], winnerdf["CDS_boundary"])

        while number_of_mutations < 2 and position in range(0, len(winnerdf["SRS_boundary"])):
                
            try: 
                fragmentedCDS = mutator_Marina(basePosition, fragmentedCDS, winnerdf, codonCoordinates)
                winnerdf["mutated?"] = True
                position += 1
                number_of_mutations += 1

            except ValueError: continue
        
        winnerdf = codonReverseFragmenter(fragmentedCDS, winnerdf, winnerdf["CDS_side"], winnerdf["CDS_boundary"])
    
    # Condition C: PAM outside of CDS
    if winnerdf["mutated?"] == False:
        fragmentedRegion, winnerdf = codonFragmenter(winnerdf, winnerdf["non_CDS_side"], winnerdf["non_CDS_boundary"])
        pos_relative_to_region = winnerdf["mutable_PAM"][0] + winnerdf["positionScore"] - winnerdf["non_CDS_boundary"][0] # formula explained in MutationLogicML.pptx
        codonNumber = codonCoordinates.at[pos_relative_to_region, 'codon'] #this is the codon number (as an index in mutableCodons)
        codon = fragmentedRegion[codonNumber] #This is the codon we want to mutate
        base = codonCoordinates.at[pos_relative_to_region, 'base'] #This is the base within that codon (1-3)
        base_to_mutate_to = [i for i in ['A', 'C', 'G', 'T'] if i != codon[base-1]][0]
        newCodon = codon[:base-1] + base_to_mutate_to + codon[base:] #add a 'T' where the old base was in this codon
        fragmentedRegion[codonNumber] = newCodon
        winnerdf = codonReverseFragmenter(fragmentedRegion, winnerdf, winnerdf["non_CDS_side"], winnerdf["non_CDS_boundary"])
        winnerdf["mutated?"] = True

    return winnerdf

In [13]:
checked_0["mutated?"] = False
mutated_0_df = find_best_mutation(checked_0.iloc[0])
display(mutated_0_df)

HAL 4


C:\Users\magd5549\AppData\Local\Temp\ipykernel_9320\141412795.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winnerdf[f"{PAM_side}"] = newHA
C:\Users\magd5549\AppData\Local\Temp\ipykernel_9320\141412795.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winnerdf["mutated?"] = True


fmin                                                                491298
fmax                                                                491320
#chr                                                                    2L
strand                                                                   -
sgRNA_sequence                                     CCAGCAGCTGGACTCGAAATGTC
Gene_ID                                                        FBgn0003963
Transcript_ID                                                  FBtr0333013
Chromosome                                                              2L
Gene_Region                                                    start_codon
Start                                                               491316
Stop                                                                491318
Strand                                                                   +
Reference_Seq            CACTGATAAAAAATACCCTAAATTATCGCAACGCAAATGGTAAAGT...
HAL                      

In [14]:
print(mutated_0_df["HAL"])
print(checked_0.iloc[0]["HAL"])

TTCTAGTCCAGCTGCTGGACCGGAGGTATATCTTCAGCTACGTCGGCATCAAGTCTGCGGAAGATCCAGATTCGCGTTCCTTTTCGCGGACATTTACCGGCGGATGCTATATCGCACTCACACACAGGCACTCACTCTCACACACACGGCGAGTATCAATTATGGGCAATCAATGCGATTACTTGTATTATTTGTATTATTGGCGGCGATTAACAAGCCTTAATG
TTCGAGTCCAGCTGCTGGACCGGAGGTATATCTTCAGCTACGTCGGCATCAAGTCTGCGGAAGATCCAGATTCGCGTTCCTTTTCGCGGACATTTACCGGCGGATGCTATATCGCACTCACACACAGGCACTCACTCTCACACACACGGCGAGTATCAATTATGGGCAATCAATGCGATTACTTGTATTATTTGTATTATTGGCGGCGATTAACAAGCCTTAATG


In [15]:
# check whether the start/stop codon and sgRNA is correct
for ind, sgRNA in sgRNAdf.iterrows():
    print(refSeqPerChromosome[sgRNA["Chromosome"]][sgRNA["Start"]-1:sgRNA["Stop"]])
    print(refSeqPerChromosome[sgRNA["Chromosome"]][sgRNA["fmin"]-1:sgRNA["fmax"]])

CAT
TGCATTCTTGTGTCCAGATCCGG
CAT
GCATTCTTGTGTCCAGATCCGGG
